In [1]:
# write the code here to test the model in real time

In [1]:
import mediapipe as mp
import numpy as np
import cv2 as cv
import os


In [2]:
mp_drawing=mp.solutions.drawing_utils
mp_holistic=mp.solutions.holistic
holistic=mp_holistic.Holistic()

In [3]:
style1=mp_drawing.DrawingSpec((71, 237, 212),1,1)
style2=mp_drawing.DrawingSpec((67, 73, 247),2,1)

In [4]:

def draw_landmarks(img,results):
    mp_drawing.draw_landmarks(img,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,style1,style2)
    mp_drawing.draw_landmarks(img,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,style1,style2)
    mp_drawing.draw_landmarks(img,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,style1,style2)
    # mp_drawing.draw_landmarks(img,results.face_landmarks,mp_holistic.FACEMESH_CONTOURS,style1,style2)

In [5]:
def extract_landmarks(results):
    # face=np.array([[landmark.x,landmark.y,landmark.z] for landmark in results.face_landmarks.landmark]).flatten()  if results.face_landmarks else np.zeros(1404)
    pose=np.array([[landmark.x,landmark.y,landmark.z , landmark.visibility] for landmark in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    rh=np.array([[landmark.x,landmark.y,landmark.z] for landmark in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(63)
    lh=np.array([[landmark.x,landmark.y,landmark.z] for landmark in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(63)

    return np.concatenate([pose,lh,rh])

In [6]:
cap=cv.VideoCapture(0)
# white=cv.resize(cv.imread('white.png'),(900,500))
while True:
    _,frame=cap.read()
    # mediapipe operations
    img=cv.cvtColor(frame,cv.COLOR_BGR2RGB)
    results=holistic.process(img)
    img=cv.cvtColor(img,cv.COLOR_RGB2BGR)
    draw_landmarks(img,results)
    landmarks=extract_landmarks(results)
    # print(landmarks)
    
    cv.imshow('frame',img)
    # cv.imshow('white',white)
    if cv.waitKey(1)==27:
        cap.release()
        break

cv.destroyAllWindows()

C:\Users\vivek chouhan\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [7]:
from tensorflow.keras.models import load_model
from tensorflow import keras

In [38]:
model=load_model('model.keras')

In [39]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_24 (Bidirect  (None, 30, 128)           124416    
 ional)                                                          
                                                                 
 batch_normalization (Batch  (None, 30, 128)           512       
 Normalization)                                                  
                                                                 
 bidirectional_25 (Bidirect  (None, 30, 256)           198144    
 ional)                                                          
                                                                 
 batch_normalization_1 (Bat  (None, 30, 256)           1024      
 chNormalization)                                                
                                                                 
 bidirectional_26 (Bidirect  (None, 64)              

In [9]:
import pickle

In [11]:
actions=pickle.load(open('new_classes.pickle','rb'))

In [23]:
actions

['afternoon',
 'blank',
 'evening',
 'fine',
 'hello',
 'how are you',
 'morning',
 'name',
 'sorry',
 'thank you',
 'welcome']

In [12]:
white.shape

(500, 900, 3)

In [26]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [27]:
scaler=MinMaxScaler()

In [41]:
sentence=[]
frames=[]

threshold=0.7  # change acc to detections

white=cv.imread("white.png")

cap=cv.VideoCapture(0)
while cap.isOpened():
    _,frame=cap.read()

    img=cv.cvtColor(frame,cv.COLOR_BGR2RGB)
    results=holistic.process(img)
    img=cv.cvtColor(img,cv.COLOR_RGB2BGR)
    draw_landmarks(img,results)
    landmarks=extract_landmarks(results)
    print(landmarks.shape)
    # draw_landmarks(image,results)
    # # landmarks=extract_landmarks(results)
    # frames.insert(0,extract_landmarks(results))
    frames.append(landmarks)
    
    if len(frames)==30:
        x=np.array(frames)

        frames.clear()
        # res_1=actions[np.argmax(model.predict(np.expand_dims(x,0))[0])]
        res_1=model.predict(np.expand_dims(x,0))[0]  # this has the array of predictions
        # sentence.append(res)
        cv.putText(img,f'{actions[np.argmax(res_1)]}',(10,20),cv.FONT_HERSHEY_COMPLEX_SMALL,1,(0,0,255),1,cv.LINE_AA)
    
    # arrangement logic of words 
        if np.max(res_1)>=threshold:  # if our predicted value is greater than the threshold
            res_2=actions[np.argmax(res_1)]  #  predicted output will be stored in res_2 eg=> thanks
            if len(sentence)>0:  # if length of sentence array is greater than 0 which means there are some words present.
                if res_2!=sentence[-1]:  # if the latest outcome is not equal to the latest predicted outcome
                    sentence.append(res_2)
                
            else: # if the sentence lenght is 0 which means its the first prediction
                sentence.append(res_2) 
            

    
    cv.putText(img,f'{actions[np.argmax(res_1)]}' if res_1.all() else '',(10,20),cv.FONT_HERSHEY_COMPLEX_SMALL,1,(0,0,255),1,cv.LINE_AA)
    cv.putText(white,' '.join(sentence),(10,20),cv.FONT_HERSHEY_COMPLEX_SMALL,1,(0,0,0),1,cv.LINE_AA)
    if cv.waitKey(1)==27:
        cap.release()
        cv.destroyAllWindows()
        break

    cv.imshow("frame",img)
    cv.imshow('ur_sentence',white)

(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
1/1 [==============================] - 0s 47ms/step
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
1/1 [==============================] - 0s 37ms/step
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
1/1 [==============================] - 0s 31ms/step
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
(258,)
1/1 

In [30]:
cap.release()